In [1]:
!huggingface-cli login
!pip install --upgrade datasets
!pip install -U bitsandbytes

import os
os.environ["WANDB_DISABLED"] = "true"



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `LLM` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM`
   ━━━━━━

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
import torch

In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="https://huggingface.co/datasets/flytech/python-codes-25k/resolve/main/python-codes-25k.json"
)

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


python-codes-25k.json:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 24813
    })
})


In [4]:
num_samples_to_keep = 3000
dataset = dataset["train"].select(range(num_samples_to_keep))
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 3000
})


In [5]:
def alpaca_prompt(example):
    instruction = example.get("instruction", "").strip()
    input_text = example.get("input", "").strip()
    output = example.get("output", "").strip()

    if input_text:
        prompt = (
            "Below is an instruction that describes a task, "
            "paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
        )
    else:
        prompt = (
            "Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
        )

    return {"text": prompt}


In [6]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token

formatted_dataset = dataset.map(alpaca_prompt)

def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=512
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [7]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
# tokenizer.pad_token = tokenizer.eos_token
# def tokenize_function(examples):
#     tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
#     tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
#     return tokenized_inputs
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [8]:
# Split the training dataset to create a validation set
train_dataset, eval_dataset = tokenized_dataset.train_test_split(test_size=0.1).values()

# Now you can use train_dataset and eval_dataset in the Trainer

In [9]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", load_in_8bit=True, device_map="auto")


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [10]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)


In [11]:
training_args = TrainingArguments(
    output_dir="./lora-llama2-python",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    push_to_hub=False
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,4.554500
20,0.457800
30,0.361000
40,0.261400
50,0.188500
60,0.185200
70,0.172000
80,0.156700
90,0.158300
100,0.164800


TrainOutput(global_step=675, training_loss=0.220758418418743, metrics={'train_runtime': 148.0569, 'train_samples_per_second': 18.236, 'train_steps_per_second': 4.559, 'total_flos': 8078729674752000.0, 'train_loss': 0.220758418418743, 'epoch': 1.0})

In [13]:
model.save_pretrained("./lora-llama3.2-python-chkpoint")
tokenizer.save_pretrained("./lora-llama3.2-python-tokens")


('./lora-llama3.2-python-tokens/tokenizer_config.json',
 './lora-llama3.2-python-tokens/special_tokens_map.json',
 './lora-llama3.2-python-tokens/tokenizer.json')

In [14]:
import os
from google.colab import files
import shutil # For zipping the folder

folder_paths = ["/content/lora-llama3.2-python-chkpoint","/content/lora-llama3.2-python-tokens"] # The path to the folder you want to zip
# output_filename = "lora-llama3.2-python-chkpoint.zip
output_filenames = ["lora-llama3.2-python-chkpoint.zip","lora-llama3.2-python-tokens.zip"] # The name of the zip file to be downloaded

for  folder_path, output_filename in zip(folder_paths, output_filenames):
  # Check if the folder exists
  if not os.path.exists(folder_path):
      print(f"Error: Folder '{folder_path}' does not exist.")
  else:
      # 1. Zip the folder
      # This creates a zip archive of the entire folder's contents.
      print(f"Zipping folder '{folder_path}' to '{output_filename}'...")
      try:
          shutil.make_archive(output_filename.replace(".zip", ""), 'zip', folder_path)
          print("Folder zipped successfully.")

          # 2. Download the zip file
          print(f"Initiating download of '{output_filename}'...")
          files.download(output_filename)
          print("Download initiated. Check your browser's downloads.")

      except Exception as e:
          print(f"An error occurred during zipping or downloading: {e}")

Zipping folder '/content/lora-llama3.2-python-chkpoint' to 'lora-llama3.2-python-chkpoint.zip'...
Folder zipped successfully.
Initiating download of 'lora-llama3.2-python-chkpoint.zip'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated. Check your browser's downloads.
Zipping folder '/content/lora-llama3.2-python-tokens' to 'lora-llama3.2-python-tokens.zip'...
Folder zipped successfully.
Initiating download of 'lora-llama3.2-python-tokens.zip'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated. Check your browser's downloads.
